Using Torchtune from Pytorch. Instructions [here](https://pytorch.org/torchtune/stable/tutorials/lora_finetune.html)

In [20]:
from torchtune.models.llama3_2 import qlora_llama3_2_3b
from torchtune.datasets import chat_dataset
from torchtune.models.llama3 import llama3_tokenizer
from torchtune.data import Message
import torch

In [21]:
device = (
    "mps" 
    if torch.backends.mps.is_available() 
    else "cuda" 
    if torch.cuda.is_available() 
    else "cpu"
)
device = torch.device(device)

In [1]:
qlora_model = qlora_llama3_2_3b(lora_attn_modules=["q_proj", "v_proj"])

import error: No module named 'triton'


In [18]:
tokenizer = llama3_tokenizer("/tmp/Llama-3.2-3B-Instruct/original/tokenizer.model")
ds = chat_dataset(
    tokenizer=tokenizer,
    source="json",
    data_files="./QLoRA/business_qlora.json",
    conversation_column="conversations",
    conversation_style="sharegpt",
)

In [ ]:
!tune run lora_finetune_single_device --config llama3_2/3B_qlora_single_device